In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.16.0 to work with agogemls


In [2]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

TitanicAutoML version: 4


titanic_model version: 1
	 Training context : Pipeline


Titanic version: 5
	 Training context : Remote Estimator
	 Regularization Rate : 0.45
	 AUC : 0.8230025183150182
	 Accuracy : 0.7985074626865671


Titanic version: 3
	 Training context : Estimator
	 Regularization Rate : 0.4
	 AUC : 0.822945283882784
	 Accuracy : 0.7947761194029851


Titanic version: 1
	 Training context : Local
	 AUC : 0.823345924908425
	 Accuracy : 0.7798507462686567




In [3]:
import joblib

model_path = ws.models['TitanicAutoML'].download(target_dir='.', exist_ok=True)
model = joblib.load(model_path)

In [4]:
model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                min_child_samples=20,
                                                                                                min_child_weight=0.001,
                                                                                                min_split_gain=0.0,
                                      

In [5]:
import pandas

test_df = pandas.read_csv('test.csv')
test =test_df.drop(['PassengerId'], axis=1)

In [6]:
output = pandas.DataFrame({
    'PassengerId': test_df['PassengerId'], 
    'Survived': model.predict(test)
})
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [28]:
import json

test_json = test[0:1].to_json()
input_json = json.dumps({"data": test_json})
input_json

#input_json = json.dumps({"data": })

'{"data": "{\\"Pclass\\":{\\"0\\":3},\\"Name\\":{\\"0\\":\\"Kelly, Mr. James\\"},\\"Sex\\":{\\"0\\":\\"male\\"},\\"Age\\":{\\"0\\":34.5},\\"SibSp\\":{\\"0\\":0},\\"Parch\\":{\\"0\\":0},\\"Ticket\\":{\\"0\\":\\"330911\\"},\\"Fare\\":{\\"0\\":7.8292},\\"Cabin\\":{\\"0\\":null},\\"Embarked\\":{\\"0\\":\\"Q\\"}}"}'

In [38]:
import pandas

data = json.loads(input_json)['data']
data = pandas.read_json(data, orient='records')
data
predictions = model.predict(data)
classnames = ['not-survived', 'survived']
predicted_classes = []
for prediction in predictions:
    predicted_classes.append(classnames[prediction])

predicted_classes

['not-survived']

In [17]:
import pandas

resultList = []
f = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/d12/code/users/marcin.szeliga/Titanic/batch-data/41.csv'
data = pandas.read_csv(f)
#print(os.path.basename(f))
p = model.predict(f1)
resultList.append("{}: {}".format(os.path.basename(f), p))

f = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/d12/code/users/marcin.szeliga/Titanic/batch-data/1.csv'
data = pandas.read_csv(f)
#print(os.path.basename(f))
p = model.predict(f1)
resultList.append("{}: {}".format(os.path.basename(f), p))
resultList

['41.csv: [1 1 0 0 1 0 0 1]', '1.csv: [1 1 0 0 1 0 0 1]']